In [14]:
import json

data = {"intents":[
    {"tag":"greeting",
     "patterns":["Hi","Hey","Is anyone there?","Hello","Wassup"],
     "responses":["Hello","Hi","Hi there"]
     },
     {"tag":"goodbye",
      "patterns":["Bye","See you later","Goodbye"],
      "responses":["See you later","Have a nice day","Bye! Come back soon"]
      },
      {"tag":"thanks",
       "patterns":["Thanks","Thank you","That's helpful","Thanks for the help"],
       "responses":["Happy to help!","You're most welcome!","My Pleasure","Anytime!"]
       },
       {"tag":"about",
        "patterns":["Who are you?","What are you?","What is this chatbot?"],
        "responses":["I'm Suresh, your favourite chat assistant","I'm Suresh, an Artificial Intelligent bot"]
        },
        {"tag":"name",
         "patterns":["What is your name?","What should I call you?","What do I call you as?","What is the name of this chatbot?"],
         "responses":["You can call me Suresh","I'm SUresh!","Just call me as Suresh"]
         },
         {"tag":"help",
          "patterns":["Can you help me?","Just give me your hand please?","Can you help?","What can you do for me?","I need a support","I need a help","Support me please"],
          "responses":["Tell me how can I assist you","Tell me your problem to assist you","Yes Sure, How can I support you?"]
          },
          {"tag":"createaccount",
           "patterns":["I need to create a new account","How to open a new account?","I want to create an account","Can you create an account?"],
           "responses":["You can just easily create a new account from our website","Just go to our website and follow the guidelines to create a new account"]
           },
           {"tag":"complaint",
            "patterns":["I have a complaint","I want to raise a complaint","There is a complaint about a service"],
            "responses":["Please provide us your complaint in order to assist you","Please mention your complaint, we will reach you and sorry for any incoveniance caused"]
            }
]}

json_data = json.dumps(data)

with open("intents.json","w") as json_file:
    json_file.write(json_data)

In [15]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [16]:
with open('intents.json') as file:
    data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [17]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [18]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences,truncating='post',maxlen=max_len)

In [19]:
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences,np.array(training_labels),epochs=epochs)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________________________________________

Epoch 1/500
2/2 [==============================] - 1s 4ms/step - loss: 2.0784 - accuracy: 0.1515
Epoch 2/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0774 - accuracy: 0.1515
Epoch 3/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0770 - accuracy: 0.1515
Epoch 4/500
2/2 [==============================] - 0s 4ms/step - loss: 2.0768 - accuracy: 0.1515
Epoch 5/500
2/2 [==============================] - 0s 5ms/step - loss: 2.0767 - accuracy: 0.1515
Epoch 6/500
2/2 [==============================] - 0s 4ms/step - loss: 2.0763 - accuracy: 0.1515
Epoch 7/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0759 - accuracy: 0.1515
Epoch 8/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0755 - accuracy: 0.1515
Epoch 9/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0750 - accuracy: 0.3333
Epoch 10/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0746 - accuracy: 0.2121
Epoch 11/500
2/2 [===========

In [20]:
model.save("chat_model")

import pickle

with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle','wb') as ecn_file:
    pickle.dump(lbl_encoder,ecn_file,protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


INFO:tensorflow:Assets written to: chat_model\assets


In [21]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama
colorama.init()
from colorama import Fore,Style,Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)

def chat():
    model = keras.models.load_model('chat_model')

    with open('tokenizer.pickle','rb') as handle:
        tokenizer  = pickle.load(handle)

    with open('label_encoder.pickle','rb') as enc:
        lbl_encoder = pickle.load(enc)

    max_len = 20

    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL,end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                                                          truncating='post',maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,np.random.choice(i['responses']))


print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)

chat()

Start messaging with the bot (type quit to stop)!
1/1 [==============================] - 0s 49ms/step
ChatBot: Hi
1/1 [==============================] - 0s 18ms/step
ChatBot: You can call me Suresh
1/1 [==============================] - 0s 17ms/step
ChatBot: Hi there
1/1 [==============================] - 0s 18ms/step
ChatBot: Hi
1/1 [==============================] - 0s 17ms/step
ChatBot: Hello
1/1 [==============================] - 0s 17ms/step
ChatBot: Hi
User: 

KeyboardInterrupt: Interrupted by user